<a href="https://colab.research.google.com/github/vin-thomas/Enron_email_classification/blob/main/Enron_bert_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imports**

In [ ]:
import os
import pandas as pd
import torch
import sklearn
from IPython.core.display import display, HTML
import glob
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
import torch.nn.functional as F

##**Installing Transformers**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 63.1 MB/s 
     |████████████████████████████████| 6.8 MB 42.0 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 70.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##**Importing Transformers and BertTokenizier**

In [ ]:
import transformers
from transformers import BertTokenizer

##**Mounting the Drive and Reading the Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/AIML Labs/@Project/working files 2b/Pre_processed/enron_csv2', index_col=[0])

##**Preprocessing the Data**

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\\', '', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'-', '', text)
    text = re.sub(r'=', '', text)
    text = re.sub(r'/', '', text)
    #text = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', '', text)
    return text

In [ ]:
df= df.drop_duplicates(subset=['Body', 'From'])
df = df[df.Folder != 'all_documents']
df = df[df.Folder != 'deleted_items']
df = df[df.Body != 'na']
df['Text']= df['Body']+df['Subject']
df.drop(['Subject', 'Body'], axis=1, inplace= True)
df['From']= df['From'].factorize()[0]
df['To']= df['To'].factorize()[0]
df['CC']= df['CC'].factorize()[0]
df['BCC']= df['BCC'].factorize()[0]
df['Name']= df['Name'].factorize()[0]
df['Text'] = df['Text'].apply(lambda x: str(x))
df.drop(['Mail_id', 'date'], axis=1, inplace=True)
df= df.reset_index(drop=True)

In [ ]:
df= df[['Name', 'From', 'To', 'CC', 'BCC', 'Text', 'Folder']]

In [ ]:
df= df.sample(60000)

In [ ]:
item_list = []
for item in df.Text:
  item = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', '', item)
  item_list.append(item)

se = pd.Series(item_list)
df['Text'] = se.values

In [ ]:
df

,Name,From,To,CC,BCC,Text,Folder
78413,91,7071,699,0,0,lblair (nonprivileged).pstrnrnhere are my fli...,meetings___general
83811,93,3193,3149,0,0,sshackle.nsfrnrncheryl and sara nnwith respec...,notes_inbox
177216,200,250,198,0,0,lcampbel (nonprivileged).pstrnrnwe received t...,inbox
21057,31,436,538,0,0,tdonoho.nsfrnrnwe are taking a poll on how yo...,notes_inbox
159128,177,1205,1937,0,0,jarnold.nsfrnrnkim:ni will not be at the may ...,discussion_threads
...,...,...,...,...,...,...,...
122605,137,9759,4183,0,0,jwolfe (nonprivileged).pstrnrnnhttp://www.bri...,09_saved
138712,151,247,280,0,0,kmann.nsfrnrnoknnnto: kathleen carnahan/na/en...,discussion_threads
71728,81,241,88,0,0,jsteffe (nonprivileged).pstrnrnjean nnthe puc...,sent_items
9118,12,1193,930,678,674,"sscott.nsfrnrni recompiled exotica.xll, so th...",notes_inbox


In [ ]:
df.nunique()

Name        207
From       8259
To         8709
CC         3802
BCC        3741
Text      59981
Folder     1000
dtype: int64

##**Split to Train/Test**

In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
train_df.shape, val_df.shape

((54000, 7), (6000, 7))

In [ ]:
train_df= train_df.reset_index(drop=True)
val_df= val_df.reset_index(drop=True)

In [ ]:
train_df['Folder']= train_df['Folder'].factorize()[0]
val_df['Folder']= val_df['Folder'].factorize()[0]

In [ ]:
n_classes= train_df.Folder.nunique()
n_classes

969

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##**Applying Standard Scaler on first 5 columns**

In [ ]:
max_len = 150
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
train_addl= train_df[['Name', 'From', 'To', 'CC', 'BCC']]
train_addl = scaler.fit_transform(train_addl)

val_addl= val_df[['Name', 'From', 'To', 'CC', 'BCC']]
val_addl = scaler.fit_transform(val_addl)

train_df = train_df.drop(['Name', 'From', 'To', 'CC', 'BCC'], axis = 1)
val_df = val_df.drop(['Name', 'From', 'To', 'CC', 'BCC'], axis = 1)

In [ ]:
train_addl

array([[-1.75319212, -0.83668645, -0.81807143, -0.3637925 , -0.36183796],
       [-0.38238884, -0.69781382, -0.62389062, -0.36798162, -0.36611829],
       [ 0.15990696,  0.15172895,  0.24669068,  2.10862495,  2.13444992],
       ...,
       [ 1.41020006,  1.43367059,  2.7226756 , -0.36798162, -0.36611829],
       [ 1.44032761,  2.86842472,  2.74853445,  5.14070857,  5.16321073],
       [-1.33140649, -0.3401116 , -0.14598075, -0.36798162, -0.36611829]])

##**Custom Dataset to return the addl columns and encodings for the text column**

In [ ]:
class BBC(Dataset):

  def __init__(self, df, addl, tokenizer, max_len):
    self.text = df.Text
    self.targets = df.Folder
    self.addl= addl
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, item):
    text = str(self.text[item])
    target = self.targets[item]
    addl_features = self.addl[item]

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors= 'pt',
      truncation= True
      
    )

    return {
      'review_text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long),
      'addl_features': torch.tensor(addl_features, dtype=torch.float)
    }

##**Function to create data_loader**

In [ ]:
def create_data_loader(df, addl, tokenizer, max_len, batch_size):
  
  ds = BBC(
    df= df,
    addl = addl,
    tokenizer=tokenizer,
    max_len=max_len    
    )

  return DataLoader(
      ds,
      batch_size=batch_size,
      shuffle = True,
      num_workers=2
      )

In [ ]:
BATCH_SIZE = 50
val_batch_size =50
train_data_loader = create_data_loader(train_df, train_addl, tokenizer, max_len, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, val_addl, tokenizer, max_len, val_batch_size)

In [ ]:
from torch import nn
from transformers import BertModel, BertForSequenceClassification

class BertClassifier(nn.Module):

  def __init__(self):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.out1 = nn.Linear(773, 1600)
    self.out2 = nn.Linear(1600, 1200)
    self.out3 = nn.Linear(1200, n_classes)
    
    for param in self.bert.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, addl_features):
    _, pooled_output = self.bert(input_ids= input_ids, 
                                 attention_mask=attention_mask,
                                 return_dict=False
                                 )
    x =  torch.cat((pooled_output,addl_features), -1)
    x = self.out1(F.relu(x))
    x = self.out2(F.relu(x))
       
    final_layer = self.out3(x)        
    
    return final_layer

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
model = BertClassifier()
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 5e-6)

In [ ]:
epochs=1
train_losses = []
train_accuracy = []
model.train()


for e in range(epochs):
  train_loss = 0
  correct = 0
  i=0
  for batch in (train_data_loader):
    i+=1
    input_ids = batch['input_ids'].to(device, dtype= torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
    labels_ = batch['targets']
    labels = torch.nn.functional.one_hot(labels_, n_classes).squeeze(1)
    labels = labels.to(device, dtype= torch.float)
    addl_features = batch['addl_features'].to(device, dtype= torch.float) 
    
 
    model.zero_grad()
 
    result = model(input_ids= input_ids, attention_mask=attention_mask, addl_features= addl_features)
    loss = criterion(result, labels_.to(device))
    train_loss +=loss.item()

    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(),1)
    optimizer.step()

    _, predicted = torch.max(result, 1)
    correct += (predicted == labels_.to(device)).sum().item()

    if i%100==0:
      print(i,')', 'loss', loss.item(), 'correct', (predicted == labels_.to(device)).sum().item())
    

  train_losses.append(train_loss/(train_df.shape[0]))
  train_accuracy.append(100 * correct/(train_df.shape[0]))
  print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(e+1,train_losses[-1], train_accuracy[-1]))

In [ ]:
val_losses = 0
val_accuracy = 0
model.eval()

i=0
for val_batch in (val_data_loader):
  i+=1
  input_ids = val_batch['input_ids'].to(device, dtype= torch.long)
  attention_mask = val_batch['attention_mask'].to(device, dtype= torch.long)
  labels_ = val_batch['targets']
  addl_features = val_batch['addl_features'].to(device, dtype= torch.float) 
  
  result = model(input_ids= input_ids, attention_mask=attention_mask, addl_features= addl_features)
  loss = criterion(result, labels_.to(device))
  val_losses +=loss.item()
 
  _, predicted = torch.max(result, 1)
  correct = (predicted == labels_.to(device)).sum().item()
  val_accuracy +=correct

  if i%100==0:
    print(i,')', 'loss', loss.item(), 'correct', (predicted == labels_.to(device)).sum().item())
     

print('Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(val_losses/val_df.shape[0], val_accuracy/val_df.shape[0]))